In [1]:
from crossreference import *
from codetimer import CodeTimer
import johncarter_squeeze as jc

In [2]:
para = jc.DEFAULT_JC_PRAM.copy()
#para['dev_mode'] = 'smpl'
para['atr_mode'] = 'ema'

with CodeTimer() as ct:
    df = jc.process_jcsqueeze('MSFT', para)



[ == default wrap codes == ] executed in 0:00:00.412956


In [4]:
df.head()

,adj_open,adj_high,adj_low,adj_close,adj_volume,bb_middle,bb_upper,bb_lower,kc_middle,kc_upper,kc_lower,macd,wavea,waveb,wavec
effective_date,,,,,,,,,,,,,,,
1986-03-13,0.058941,0.067609,0.058941,0.064720,1031788800.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-03-14,0.064720,0.068187,0.064720,0.067031,308160000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-03-17,0.067031,0.068765,0.067031,0.068187,133171200.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-03-18,0.068187,0.068765,0.065876,0.066454,67766400.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-03-19,0.066454,0.067031,0.064720,0.065298,47894400.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
df['is_squeeze'] = (df.bb_lower >= df.kc_lower) & (df.bb_upper <= df.kc_upper)     # squeeze indicator for color
df['squeeze_line'] = (df.is_squeeze == False) & (df.is_squeeze.shift(1) == True)

dflines = df[df.squeeze_line==True]
dflines.head()

,adj_open,adj_high,adj_low,adj_close,adj_volume,bb_middle,bb_upper,bb_lower,kc_middle,kc_upper,kc_lower,macd,wavea,waveb,wavec,squeeze,squeezee,line,is_squeeze,squeeze_line
effective_date,,,,,,,,,,,,,,,,,,,,
1986-04-14,0.066454,0.067609,0.066454,0.067031,12153600.00,0.064113,0.067955,0.060272,0.064522,0.068765,0.060278,NaN,NaN,NaN,NaN,False,False,True,False,True
1986-06-02,0.080900,0.080900,0.078589,0.078589,19728000.00,0.074341,0.079012,0.069671,0.074589,0.079039,0.070138,0.000412,NaN,NaN,NaN,False,False,True,False,True
1986-09-22,0.068765,0.068765,0.068187,0.068187,4492800.00,0.067580,0.071707,0.063454,0.067868,0.071893,0.063842,0.000109,0.000502,-0.000336,NaN,False,False,True,False,True
1986-10-08,0.071932,0.074844,0.071076,0.073966,119606400.00,0.067637,0.072459,0.062815,0.068184,0.073386,0.062981,0.000691,0.001676,0.001082,NaN,False,False,True,False,True
1987-01-07,0.118461,0.124817,0.118461,0.124817,60998400.00,0.112380,0.120553,0.104208,0.113041,0.120642,0.105439,0.000581,-0.000351,0.000513,NaN,False,False,True,False,True


In [118]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import datetime

def drawchart(symbol, df):
    
    # enrich the data - macd up/down, bb_low > kc_low, bb_high < kc_high
    df['macd_up'] = df.macd > df.macd.shift(-1)
    df['squeeze_value'] = 0                                                             # squeeze value set to 0 to show the marker on zero line
    df['is_squeeze'] = (df.bb_lower >= df.kc_lower) & (df.bb_upper <= df.kc_upper)     # squeeze indicator for color
    df['squeeze_line'] = (df.is_squeeze == False) & (df.is_squeeze.shift(1) == True)
    
    style_bollingerbands_middle = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'dot')
    style_bollingerbands_upper = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'line')
    style_bollingerbands_lower = dict( color = ('rgb(91, 154, 255)'), width = 1, dash = 'line')

    style_kc_middle = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'dot')
    style_kc_upper = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'line')
    style_kc_lower = dict( color = ('rgb(244, 118, 73)'), width = 1, dash = 'line')

    trace_candle = go.Candlestick(x=df.index, open=df.adj_open, high=df.adj_high, low=df.adj_low, close=df.adj_close, 
                        showlegend=False, name='close price',
                        increasing=dict(line=dict(color= '#586b59')), #b1e2b6
                        decreasing=dict(line=dict(color= '#51150e'))
                        ,hoverinfo='y'
                    )

    '''
    # OHLC chart for stock price, hide hover info to make UI more readable
    trace_ohlc = go.Ohlc(x=df.index, open=df.adj_open, high=df.adj_high, low=df.adj_low, close=df.adj_close, 
                         showlegend=False, name='close price',
                         increasing=dict(line=dict(color= '#b1e2b6')),
                         decreasing=dict(line=dict(color= '#51150e')),
                         hoverinfo='none'
                        )
    '''

    trace_bb_middle = go.Scatter(x=df.index, y=df.bb_middle, name='BB Middle', legendgroup='Bollinger Bands',
                                 showlegend=False, opacity=0.5, hoverinfo='none',
                                 line = style_bollingerbands_middle
                                )
    trace_bb_upper = go.Scatter(x=df.index, y=df.bb_upper, name='BB Upper', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_upper
                               )
    trace_bb_lower = go.Scatter(x=df.index, y=df.bb_lower, name='BB Lower', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_lower
                               )
    
    trace_kc_middle = go.Scatter(x=df.index, y=df.kc_middle, name='KC Middle', legendgroup='Keltner Channels 2ATR',
                                 showlegend=False, opacity=0.5, hoverinfo='none',
                                 line = style_kc_middle
                                )
    trace_kc_upper = go.Scatter(x=df.index, y=df.kc_upper, name='KC 2ATR Upper', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc_upper
                                    )
    trace_kc_lower = go.Scatter(x=df.index, y=df.kc_lower, name='KC 2ATR Lower', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc_lower
                                    )
    '''
    # create volume bar chart on the bottom
    tarce_volume = go.Bar(x=df.index, y=df.adj_volume, name='volume', showlegend=False, marker=dict(color='rgb(87, 91, 130)'),
                          yaxis = 'y2'
                         )
    '''

    # color dict for MACD
    macd_colordict = []
    squeeze_colordict = []
    for index, row in df.iterrows():
        # MACD
        color_macd = '#029107' if (row['macd']>=0 and row['macd_up']==True) else \
                    ('#0599b7' if (row['macd']>=0 and row['macd_up']==False) else \
                    ('#871001' if (row['macd']<0 and row['macd_up']==False) else '#d35004' ))
        macd_colordict.append(color_macd)
        # squeeze
        color_squeeze = '#022ff7' if row['is_squeeze'] else '#404241'  #f21104 for red
        squeeze_colordict.append(color_squeeze)


    trace_macd = go.Bar(x=df.index, y=df.macd, name='MACD(12/26/9)', showlegend=False, 
                        marker=dict(color=macd_colordict),
                        yaxis = 'y3'
                       )

    trace_squeeze = go.Scatter(x=df.index, y=df.squeeze_value, name='', showlegend=False, 
                               mode = 'markers', marker=dict(size=6, color=squeeze_colordict), hoverinfo='none',
                               yaxis = 'y3'
                               )

    trace_wave_a = go.Bar(x=df.index, y=df.wavea, name='WAVE A(8/34/34)', showlegend=False, 
                    marker=dict(color=['#029107' if val>=0 else '#871001' for val in df.wavea]),
                    yaxis = 'y4'
                   )

    trace_wave_b = go.Bar(x=df.index, y=df.waveb, name='WAVE B(8/89/89)', showlegend=False, 
                marker=dict(color=['#029107' if val>=0 else '#871001' for val in df.waveb]),
                yaxis = 'y5'
               )
    trace_wave_c = go.Bar(x=df.index, y=df.wavec, name='WAVE C(8/144/144)', showlegend=False, 
                    marker=dict(color=['#029107' if val>=0 else '#871001' for val in df.wavec]),
                    yaxis = 'y6'
                   )

    # collect all traces show on the chart
    data = [trace_candle, trace_bb_middle, trace_kc_middle,
            trace_bb_upper, trace_bb_lower, trace_kc_upper, trace_kc_lower, 
            #tarce_volume, 
            trace_macd, trace_squeeze, trace_wave_a, trace_wave_b, trace_wave_c]
    
    # fixrange=True, cannot zoom in, useful for y axis for the small indicators
    # fixrange=Fasle with range setting, used in the main chart so it could be zoomed in
    # define xaxis template to shorten layout
    axis_template = dict(type="date", showgrid=True, zeroline=True, showline=True, 
                         rangeslider=dict(visible=False), fixedrange=False,
                         range = [datetime.datetime(2017, 4, 1),
                            datetime.datetime(2018, 1, 1)]
                     )

    # generate shapes for squeeze line
    squeezeline_style = {'color':'rgb(30,60,30)', 'width':1, 'dash':'dot'}
    
    squeezelines = []
    dflines = df[df.squeeze_line==True]
    for idx, row in dflines.iterrows():
        squeezeline = {'type':'line', 'y0':0, 'y1':1, 'xref':'x', 'yref':'paper', 'line':squeezeline_style}
        squeezeline['x0'] = squeezeline['x1'] = idx.strftime('%Y-%m-%d')
        squeezelines.append(squeezeline)
    
    layout = go.Layout(
        title = '{} - John Carter TTM Squeeze'.format(symbol),
        # put legend in left/top coner
        legend = dict(x=0.05, y=1.0),
        height=1050,
        margin = go.Margin(l=80,r=30,t=50,b=100),
        paper_bgcolor='#000', plot_bgcolor='#000',

        xaxis = axis_template,

        # all yaxis domains (leave gap between y and below to show the x axis labels
        yaxis = dict(domain=[0.35,1], autorange=False, fixedrange=False, range=[60,90]),
        # yaxis2 = dict(domain=[0.5,0.5], visible=False),
        yaxis3 = dict(domain=[0.225,0.325], fixedrange=True, title='MACD'),
        yaxis4 = dict(domain=[0.15,0.225], fixedrange=True, visible=True, title="WAVEA"),
        yaxis5 = dict(domain=[0.075,0.15], fixedrange=True, visible=True, title="WAVEB"),
        yaxis6 = dict(domain=[0,0.075], fixedrange=True, visible=True, title="WAVEC"),
        
        shapes = squeezelines
    )

#        shapes = [{'x0':'2017-10-10', 'x1':'2017-10-10', 'y0':0, "y1":1, 'xref':'x', 'yref':'paper', 
#               'line':{'color':'rgb(30,30,30)', 'width':1}}]

    
    config = {'showLink':False, 'modeBarButtonsToRemove': ['sendDataToCloud','zoom2d','select2d','lasso2d']}
    
    fig = go.Figure(data=data, layout=layout )
    # standalone chart
    plotly.offline.plot(fig, filename="{} - John Carter Squeeze Chart.html".format(symbol), config=config)



In [119]:
len(df)

8054

In [122]:
drawchart('AMZN', df[7000:])

C:\Program Files\Python36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Program Files\Python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Program Files\Python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html